In [ ]:
# Importar e instalar dependencias
import pandas as pd
import numpy as np
import re
from collections import Counter

# CARGA DE DATOS

In [ ]:
from sklearn.model_selection import train_test_split
!pip install gdown
# Descarga el archivo del dataset de drive usando gdown
url = 'https://drive.google.com/file/d/1pBs7QN2yZa8aV_hE00oLP6s0cyHm9QZI/view?usp=drive_link'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset en inglés usando pandas
sd_E = pd.read_csv('/content/Suicide_Detection_clean_bilingual.csv')

# Descarga el archivo del dataset en español de drive usando gdown
url = 'https://drive.google.com/file/d/19Nd5ulojA08fjSr96lHsbQRuokFiF4Sl/view?usp=drive_link'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset en español usando pandas
sd_S = pd.read_csv('/content/Suicide_Detection_clean_translated_102224.csv')

# Combina los dos datasets
combined_data = pd.concat([sd_E, sd_S], ignore_index=True)

# Shuffle the combined dataset
combined_data = combined_data.sample(frac=1).reset_index(drop=True)

# Split the combined dataset into training and testing sets
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42)

In [ ]:
combined_data.info()
combined_data.head()

# MODELADO, ENTRENAMIENTO Y EVALUACIÓN

In [ ]:
!pip install transformers torch evaluate datasets
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig, EarlyStoppingCallback
import torch
from sklearn.preprocessing import LabelEncoder
import evaluate
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
# Configuración personalizada del modelo DistilBERT
model_name = 'distilbert-base-multilingual-cased'
custom_config = DistilBertConfig.from_pretrained(model_name)
custom_config.hidden_dropout_prob = 0.30
custom_config.attention_probs_dropout_prob = 0.42
custom_config.num_labels = 2

# Cargar el tokenizador y el modelo DistilBERT con la configuración personalizada
tokenizer_bert = DistilBertTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
model = DistilBertForSequenceClassification.from_pretrained(model_name, config=custom_config)

# Crear un encoder para las etiquetas
label_encoder = LabelEncoder()

# Ajustar el encoder a las etiquetas y transformar
train_data['class'] = label_encoder.fit_transform(train_data['class'])
test_data['class'] = label_encoder.transform(test_data['class'])

def encode_data(examples):
# Convert the 'text' column to a list of strings
    texts = examples['text']
    if isinstance(texts, (pd.Series, np.ndarray)):
        texts = texts.tolist()  # Convert Series or array to list
    elif isinstance(texts, str):
        texts = [texts]         # Convert single string to list
    return tokenizer_bert(texts, truncation=True, padding='max_length', max_length=128)

#return tokenizer_bert(examples['text'], truncation=True, padding='max_length', max_length=128)

# Convertir los datos a datasets de Hugging Face
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Aplicar la codificación a los datasets
train_dataset = train_dataset.map(encode_data, batched=True)
test_dataset = test_dataset.map(encode_data, batched=True)

# Renombrar la columna 'class' a 'labels'
train_dataset = train_dataset.rename_column('class', 'labels')
test_dataset = test_dataset.rename_column('class', 'labels')

# Configurar el formato de los datasets
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, config=custom_config)
# Configurar argumentos de entrenamiento con los nuevos hiperparámetros
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2.9e-05,
    warmup_ratio=0.08,
    weight_decay=0.06,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to = 'none'
)


# Función para calcular métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
eval_results = trainer.evaluate()

# Imprimir los resultados de evaluación
print(eval_results)

# Realizar predicciones en el conjunto de prueba
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

accuracy = accuracy_score(test_dataset['labels'], predicted_labels)
precision = precision_score(test_dataset['labels'], predicted_labels, average='weighted')
recall = recall_score(test_dataset['labels'], predicted_labels, average='weighted')
f1 = f1_score(test_dataset['labels'], predicted_labels, average='weighted')


print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
# Guardar solo los pesos (más común y recomendado)
torch.save(model.state_dict(), 'pesos_modelo_ult.pth')

In [ ]:
!zip -r /content/pesos_modelo_ult.pth.zip /content/pesos_modelo_ult.pth

In [ ]:
from google.colab import files
# Después de guardar el modelo, ejecuta:
files.download('pesos_modelo_ult.pth.zip')